In [61]:
import numpy as np
import csv
import sys
import os

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [62]:
def plot_background_fct(send_data, recv_data, label, do_plot=True):
    # diffs in miliseconds
    diffs = (recv_data - send_data) * 1000
    if do_plot:
        fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
        ax1.plot(diffs, label="background FCT"+label) 
        ax1.set_title("Background FCT")
        ax1.legend(loc='upper left')
        ax1.set_ylabel('Miliseconds')
        ax1.set_xlabel('Request ID')
        ax1.grid(which='major', linestyle="-")
        ax1.grid(which='minor', linestyle='--')

        fig.tight_layout()   

        fig.savefig("Queuing delay vs Time.png", format="png")
        fig.show()
    return np.mean(diffs), np.percentile(diffs,99)

In [63]:
def plot_foreground_fct(send_data, recv_data, label, do_plot=True):
    # diffs in micrseconds
    diffs = (recv_data - send_data) * 1000 * 1000
    if do_plot:
        fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
        ax1.plot(diffs, label="background FCT"+label) 
        ax1.set_title("Foreground FCT")
        ax1.legend(loc='upper left')
        ax1.set_ylabel('Microseconds')
        ax1.set_xlabel('Request ID')
        ax1.grid(which='major', linestyle="-")
        ax1.grid(which='minor', linestyle='--')

        fig.tight_layout()   

        fig.savefig("Queuing delay vs Time.png", format="png")
        fig.show()
    return np.mean(diffs), np.percentile(diffs,99)

In [64]:
def plot_queue_size(data, label, do_plot=True):
    # http://www.mathcs.emory.edu/~cheung/Courses/558-old/Syllabus/90-NS/trace.html
    # Columns are: Time(s)-fromNode-toNode-SizeB-SizePack-Packsarrived(in interval)
    # -departed - dropped -3: same in bytes
    times = data[:,0]
    q_size_B, q_size_P = data[:,3], data[:,4]
    q_size_B = np.trim_zeros(q_size_B, trim='b')
    q_size_P = np.trim_zeros(q_size_P, trim='b')
    times = times[:q_size_B.shape[0]]
    
    q_size_B = np.trim_zeros(q_size_B, trim='f')
    q_size_P = np.trim_zeros(q_size_P, trim='f')
    
    times = times[times.shape[0] - q_size_B.shape[0]:]
    
    if do_plot:
        fig, (ax1,ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10,10))
        ax1.plot(times, q_size_B, label="q_size in bytes "+label) 
        ax1.set_title("Q size vs Time")
        ax1.legend(loc='upper left')
        ax1.set_ylabel('Bytes')
        ax1.set_xlabel('Seconds')
        ax1.grid(which='major', linestyle="-")
        ax1.grid(which='minor', linestyle='--')

        ax2.plot(times, q_size_P, label="q_size in packets "+label) 
        ax2.set_title("Q size vs Time")
        ax2.legend(loc='upper left')
        ax2.set_ylabel('Packets')
        ax2.set_xlabel('Seconds')
        ax2.grid(which='major', linestyle="-")
        ax2.grid(which='minor', linestyle='--')

        fig.tight_layout()   

        fig.savefig("Q_size vs Time.png", format="png")
        fig.show()
    #avg_q_occup = np.mean(q_size_B)
    #print("Mon_mean: " + str(avg_q_occup))
    q_size_B = q_size_B/1000.0
    return np.mean(q_size_B), np.mean(q_size_P), np.percentile(q_size_B,99), np.percentile(q_size_P, 99)

In [65]:
def plot_queuing_time(data, label, link_speed, do_plot=True):
    """
    Link speed in Gbps
    results in microseconds
    """
    times = data[:,0]
    q_size_B, q_size_P = data[:,3], data[:,4]
    q_size_B = np.trim_zeros(q_size_B, trim='b')
    times = times[:q_size_B.shape[0]]
    q_size_B = np.trim_zeros(q_size_B, trim='f')
    times = times[times.shape[0] - q_size_B.shape[0]:]
    
    link_speed = link_speed * 1000.0 * 1000.0 * 1000.0
    q_time = q_size_B/(link_speed/8.0) * 1000.0 * 1000.0
    
    if do_plot:
        fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
        ax1.plot(times, q_time, label="queuing delay "+label) 
        ax1.set_title("Queuing delay vs Time")
        ax1.legend(loc='upper left')
        ax1.set_ylabel('Queuing microseconds')
        ax1.set_xlabel('Simulation Seconds')
        ax1.grid(which='major', linestyle="-")
        ax1.grid(which='minor', linestyle='--')

        fig.tight_layout()   

        fig.savefig("Queuing delay vs Time.png", format="png")
        fig.show()
    return np.mean(q_time), np.percentile(q_time, 99)

In [66]:
def print_results():
    print("Mean Q length in Kbytes {0} and in packets {1}\n99th percentile in Kbytes {2} and in packets {3}\n".format(bytes_mean, pkts_mean, bytes_99, pkts_99))
    print("Mean delay in us {0} and 99th percentile delay in us {1}\n".format(delay_mean, delay_99))
    print("Background FCT (ms): mean: {0}, 99th %: {1}\n".format(bkg_mean, bkg_99))
    print("Foreground FCT (us): mean: {0}, 99th %: {1}\n".format(frg_mean, frg_99))
    
    #print("Mean thrpt in Mbps {0} and 99th percentile thrpt in Mbps {1}".format(thrpt_mean, thrpt_99))    

# Paper reproduction
## 20% bottlenecked link load 

In [67]:
results_folders = ["/TCP/", "/DCTCP30/", "/DCTCP6_pacer/", "/DCTCP_pacer_PQ/"]
latency = dict()
bkg_fct = dict()
frg_fct = dict()

for folder in results_folders:
    latency[folder] = []
    bkg_fct[folder] = []
    frg_fct[folder] = []

In [68]:
simul_folder = "nofanout_20_9flows"

for folder in results_folders:
    path = simul_folder + folder
    q_mon = np.genfromtxt(path+"q_mon", delimiter=" ")
    bytes_mean, pkts_mean, bytes_99, pkts_99 = plot_queue_size(q_mon, folder, do_plot=False)
    delay_mean, delay_99 = plot_queuing_time(q_mon, folder, 1, do_plot=False)
    latency[folder].append((delay_mean, delay_99))
    
    bkg_send = np.genfromtxt(path+"send_times|background.csv", delimiter=" ")
    bkg_recv = np.genfromtxt(path+"rec_times|background.csv", delimiter=" ")
    bkg_mean, bkg_99 = plot_background_fct(bkg_send, bkg_recv, folder, do_plot=False)
    bkg_fct[folder].append((bkg_mean,bkg_99))
    
    frg_send = np.genfromtxt(path+"send_times|foreground.csv", delimiter=" ")
    frg_recv = np.genfromtxt(path+"rec_times|foreground.csv", delimiter=" ")
    frg_mean, frg_99 = plot_foreground_fct(frg_send, frg_recv, folder, do_plot=False)
    frg_fct[folder].append((frg_mean,frg_99))
    print(folder)
    print_results()

/TCP/
Mean Q length in Kbytes 10.774666688855316 and in packets 10.885744478247267
99th percentile in Kbytes 285.5532155044199 and in packets 288.54015191138933

Mean delay in us 86.19733351084253 and 99th percentile delay in us 2284.425724035359

Background FCT (ms): mean: 101.79763414446701, 99th %: 258.93551357355244

Foreground FCT (us): mean: 141.7319050942629, 99th %: 2387.537782686878

/DCTCP30/
Mean Q length in Kbytes 0.8927973556867697 and in packets 0.902016540756956
99th percentile in Kbytes 16.640003356381243 and in packets 16.810790949444367

Mean delay in us 7.142378845494157 and 99th percentile delay in us 133.12002685104994

Background FCT (ms): mean: 103.08987722794909, 99th %: 276.4441299828415

Foreground FCT (us): mean: 59.696303975787615, 99th %: 791.951203912334

/DCTCP6_pacer/
Mean Q length in Kbytes 0.5208935263417955 and in packets 0.526289889906458
99th percentile in Kbytes 15.527113805865126 and in packets 15.685673939583097

Mean delay in us 4.16714821073436

## 40% bottlenecked link load 

In [69]:
simul_folder = "nofanout_40_9flows"

for folder in results_folders:
    path = simul_folder + folder
    q_mon = np.genfromtxt(path+"q_mon", delimiter=" ")
    bytes_mean,pkts_mean,bytes_99,pkts_99 = plot_queue_size(q_mon, folder, do_plot=False)
    delay_mean, delay_99 = plot_queuing_time(q_mon, folder, 1, do_plot=False)
    latency[folder].append((delay_mean, delay_99))
    
    
    bkg_send = np.genfromtxt(path+"send_times|background.csv", delimiter=" ")
    bkg_recv = np.genfromtxt(path+"rec_times|background.csv", delimiter=" ")
    bkg_mean, bkg_99 = plot_background_fct(bkg_send, bkg_recv, folder, do_plot=False)
    bkg_fct[folder].append((bkg_mean,bkg_99))
    
    
    frg_send = np.genfromtxt(path+"send_times|foreground.csv", delimiter=" ")
    frg_recv = np.genfromtxt(path+"rec_times|foreground.csv", delimiter=" ")
    frg_mean, frg_99 = plot_foreground_fct(frg_send, frg_recv, folder, do_plot=False)
    frg_fct[folder].append((frg_mean,frg_99))
    
    
    
    print(folder)
    print_results()

/TCP/
Mean Q length in Kbytes 44.0350871854736 and in packets 44.49438107921128
99th percentile in Kbytes 426.53155550045904 and in packets 430.86176231899356

Mean delay in us 352.2806974837888 and 99th percentile delay in us 3412.252444003673

Background FCT (ms): mean: 134.22350802345144, 99th %: 447.762181889981

Foreground FCT (us): mean: 430.20916449196415, 99th %: 3560.0888624011154

/DCTCP30/
Mean Q length in Kbytes 3.3571748713879552 and in packets 3.392247328834531
99th percentile in Kbytes 25.561687354265544 and in packets 25.82014997112857

Mean delay in us 26.857398971103642 and 99th percentile delay in us 204.49349883412435

Background FCT (ms): mean: 139.15430947827215, 99th %: 427.55805534553

Foreground FCT (us): mean: 87.5904377335351, 99th %: 912.9921518683655

/DCTCP6_pacer/
Mean Q length in Kbytes 1.9031507946465418 and in packets 1.9230807639486158
99th percentile in Kbytes 21.249498025095686 and in packets 21.464307707570956

Mean delay in us 15.22520635717233 an

## 60% bottlenecked link load

In [70]:
simul_folder = "nofanout_60_9flows"

for folder in results_folders:
    path = simul_folder + folder
    q_mon = np.genfromtxt(path+"q_mon", delimiter=" ")
    bytes_mean,pkts_mean,bytes_99,pkts_99 = plot_queue_size(q_mon, folder, do_plot=False)
    delay_mean, delay_99 = plot_queuing_time(q_mon, folder, 1, do_plot=False)
    latency[folder].append((delay_mean, delay_99))
    
    
    bkg_send = np.genfromtxt(path+"send_times|background.csv", delimiter=" ")
    bkg_recv = np.genfromtxt(path+"rec_times|background.csv", delimiter=" ")
    bkg_mean, bkg_99 = plot_background_fct(bkg_send, bkg_recv, folder, do_plot=False)
    bkg_fct[folder].append((bkg_mean,bkg_99))
    
    
    frg_send = np.genfromtxt(path+"send_times|foreground.csv", delimiter=" ")
    frg_recv = np.genfromtxt(path+"rec_times|foreground.csv", delimiter=" ")
    frg_mean, frg_99 = plot_foreground_fct(frg_send, frg_recv, folder, do_plot=False)
    frg_fct[folder].append((frg_mean,frg_99))
    
    
    print(folder)
    print_results()

/TCP/
Mean Q length in Kbytes 101.46689781627384 and in packets 102.5355332982317
99th percentile in Kbytes 464.75284139133 and in packets 469.4904869718644

Mean delay in us 811.7351825301907 and 99th percentile delay in us 3718.02273113064

Background FCT (ms): mean: 206.2282459162474, 99th %: 824.2336581929637

Foreground FCT (us): mean: 869.3429159026326, 99th %: 3791.5498266876584

/DCTCP30/
Mean Q length in Kbytes 7.203022174220621 and in packets 7.278970506775793
99th percentile in Kbytes 29.020916144586927 and in packets 29.323904141235605

Mean delay in us 57.62417739376497 and 99th percentile delay in us 232.16732915669542

Background FCT (ms): mean: 219.18406769517912, 99th %: 729.3707723005915

Foreground FCT (us): mean: 117.08309767296898, 99th %: 843.4728640018828

/DCTCP6_pacer/
Mean Q length in Kbytes 3.750809468351261 and in packets 3.790448781035403
99th percentile in Kbytes 23.568432851359034 and in packets 23.806946440900948

Mean delay in us 30.00647574681009 and 9

In [71]:
print(latency)
print("------------------")
print(bkg_fct)
print("------------------")
print(frg_fct)

{'/TCP/': [(86.19733351084253, 2284.425724035359), (352.2806974837888, 3412.252444003673), (811.7351825301907, 3718.02273113064)], '/DCTCP30/': [(7.142378845494157, 133.12002685104994), (26.857398971103642, 204.49349883412435), (57.62417739376497, 232.16732915669542)], '/DCTCP6_pacer/': [(4.167148210734364, 124.21691044692099), (15.22520635717233, 169.99598420076552), (30.00647574681009, 188.54746281087228)], '/DCTCP_pacer_PQ/': [(1.4286854686168238, 60.052693149876575), (2.8381942744212583, 57.836609872545765), (12.856125942355781, 106.0808379412392)]}
------------------
{'/TCP/': [(101.79763414446701, 258.93551357355244), (134.22350802345144, 447.762181889981), (206.2282459162474, 824.2336581929637)], '/DCTCP30/': [(103.08987722794909, 276.4441299828415), (139.15430947827215, 427.55805534553), (219.18406769517912, 729.3707723005915)], '/DCTCP6_pacer/': [(105.03180388403679, 275.4308487525236), (144.6957124180406, 429.91086972586567), (228.5973137790125, 732.0288082414718)], '/DCTCP_p

In [72]:
# fig, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(16,10))
# for i, setup in enumerate(results_folders):
#     ax1.bar(i-0.4, latency[setup][0][0], width=0.1, log=True, color="blue")
#     ax1.bar(i-0.3, latency[setup][0][1], width=0.1, log=True, color="blue")
#     ax1.bar(i-0.1, latency[setup][1][0], width=0.1, log=True, color="red")
#     ax1.bar(i, latency[setup][1][1], width=0.1, log=True, color="red")
#     ax1.bar(i+0.2, latency[setup][2][0], width=0.1, log=True, color="green")
#     ax1.bar(i+0.3, latency[setup][2][1], width=0.1, log=True, color="green")
# ax1.set_title("Switch Latency")
# ax1.legend(loc='upper left')
# ax1.set_ylabel('Queuing miliseconds')
# ax1.set_xlabel('Scheme')
# ax1.grid(which='major', linestyle="-")
# #ax1.grid(which='minor', linestyle='--')

# fig.tight_layout()   

# fig.savefig("Queuing delay vs Time.png", format="png")
# fig.show()